<a href="https://colab.research.google.com/github/antoniovfonseca/agentic-ai-global-lulc/blob/main/transition-matrix-gee-glance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!pip install geemap -q

In [23]:
# Import libraries
import pandas as pd
import ee
import os
import sys
import geemap


# Define repository details
repo_url = 'https://github.com/antoniovfonseca/agentic-ai-global-lulc.git'
repo_name = 'agentic-ai-global-lulc'

# Clone or update the repository
if not os.path.exists(repo_name):
    print(f"Cloning {repo_name}...")
    !git clone {repo_url}
else:
    print(f"Updating {repo_name}...")
    !cd {repo_name} && git pull

# Add the notebooks directory to system path to allow imports
notebooks_path = os.path.join(repo_name, 'notebooks')
if notebooks_path not in sys.path:
    sys.path.append(os.path.abspath(notebooks_path))
    print(f"Added {notebooks_path} to system path.")

import importlib
import utils

importlib.reload(utils)
print("Utils module loaded successfully.")

Updating agentic-ai-global-lulc...
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 4 (delta 2), reused 0 (delta 0), pack-reused 0 (from 0)
Unpacking objects: 100% (4/4), 2.09 KiB | 1.05 MiB/s, done.
From https://github.com/antoniovfonseca/agentic-ai-global-lulc
   20384cb..c3365e0  main       -> origin/main
Updating 20384cb..c3365e0
Fast-forward
 notebooks/utils.py | 101 +++++++++++++++++++++++++++++++++++++++++++++++++++++
 1 file changed, 101 insertions(+)
Added agentic-ai-global-lulc/notebooks to system path.
Utils module loaded successfully.


In [24]:
import ee
import geemap
import pandas as pd
import utils  # Certifique-se que o utils.py está na mesma pasta ou no sys.path

# Autenticação e Inicialização
try:
    ee.Initialize(project='ee-antoniovicgf')
except Exception as e:
    ee.Authenticate()
    ee.Initialize(project='ee-antoniovicgf')

In [22]:
# 1. Define the full temporal range from 2001 to 2019
full_year_list = list(range(2001, 2020))

# 2. Execute the batch calculation for all transitions
# This returns a dictionary with 18 annual matrices and 1 long-term matrix
all_transition_matrices = utils.calculate_global_transition_matrix(
    year_list=full_year_list,
    scale=300
)

# 3. Example: Display the long-term transition (2001 to 2019)
print("Long-term transition matrix (2001-2019):")
display(all_transition_matrices["2001_2019"])

# 4. Example: Access a specific annual change (e.g., 2005 to 2006)
# display(all_transition_matrices["2005_2006"])

Successfully computed: 2001_2002
Successfully computed: 2002_2003
Successfully computed: 2003_2004
Successfully computed: 2004_2005
Successfully computed: 2005_2006
Successfully computed: 2006_2007


Successfully computed: 2007_2008


KeyboardInterrupt: 

In [25]:
# 1. Define the full range of years (2001 to 2019)
# This will result in 18 annual transitions + 1 long-term transition
years_to_process = list(range(2001, 2020))

# 2. Trigger the export tasks via the updated utils function
# Scale=30 ensures native resolution; drive_folder is where CSVs will appear
tasks_list = utils.export_global_transition_tasks(
    year_list=years_to_process,
    drive_folder="GLANCE_Global_Transitions_30m",
    scale=30
)

# 3. Quick overview of the triggered tasks
print(f"\nSuccessfully triggered {len(tasks_list)} export tasks.")
print("You can monitor progress in the GEE Tasks tab or via code.")

Task started for transition_2001_2002 (Scale: 30m)
Task started for transition_2002_2003 (Scale: 30m)
Task started for transition_2003_2004 (Scale: 30m)
Task started for transition_2004_2005 (Scale: 30m)
Task started for transition_2005_2006 (Scale: 30m)
Task started for transition_2006_2007 (Scale: 30m)
Task started for transition_2007_2008 (Scale: 30m)
Task started for transition_2008_2009 (Scale: 30m)
Task started for transition_2009_2010 (Scale: 30m)
Task started for transition_2010_2011 (Scale: 30m)
Task started for transition_2011_2012 (Scale: 30m)
Task started for transition_2012_2013 (Scale: 30m)
Task started for transition_2013_2014 (Scale: 30m)
Task started for transition_2014_2015 (Scale: 30m)
Task started for transition_2015_2016 (Scale: 30m)
Task started for transition_2016_2017 (Scale: 30m)
Task started for transition_2017_2018 (Scale: 30m)
Task started for transition_2018_2019 (Scale: 30m)
Task started for transition_2001_2019 (Scale: 30m)

Successfully triggered 19 expo

In [27]:
# Optional cell to monitor the status of the tasks
for task in tasks_list:
    status = task.status()
    print(f"Task: {status['description']} | Status: {status['state']}")

Task: transition_2001_2002 | Status: RUNNING
Task: transition_2002_2003 | Status: RUNNING
Task: transition_2003_2004 | Status: READY
Task: transition_2004_2005 | Status: READY
Task: transition_2005_2006 | Status: READY
Task: transition_2006_2007 | Status: READY
Task: transition_2007_2008 | Status: READY
Task: transition_2008_2009 | Status: READY
Task: transition_2009_2010 | Status: READY
Task: transition_2010_2011 | Status: READY
Task: transition_2011_2012 | Status: READY
Task: transition_2012_2013 | Status: READY
Task: transition_2013_2014 | Status: READY
Task: transition_2014_2015 | Status: READY
Task: transition_2015_2016 | Status: READY
Task: transition_2016_2017 | Status: READY
Task: transition_2017_2018 | Status: READY
Task: transition_2018_2019 | Status: READY
Task: transition_2001_2019 | Status: READY


In [ ]:
import time
from tqdm.notebook import tqdm

# 1. Initialize the progress bar with the total number of tasks
pbar = tqdm(total=len(tasks_list), desc="Exporting GLANCE Matrices")

# 2. Track which tasks have already been counted as finished
completed_task_ids = set()

# 3. Monitor loop: runs until all tasks are COMPLETED or FAILED
while len(completed_task_ids) < len(tasks_list):
    for task in tasks_list:
        # 4. Skip if the task was already processed in previous iterations
        if task.id in completed_task_ids:
            continue

        # 5. Check current status from GEE server
        status = task.status()
        state = status['state']

        # 6. Update progress bar if task reaches a terminal state
        if state in ['COMPLETED', 'FAILED', 'CANCELLED']:
            completed_task_ids.add(task.id)
            pbar.update(1)

            if state == 'FAILED':
                print(f"\nTask {status['description']} failed: {status.get('error_message')}")

    # 7. Wait 30 seconds before polling the server again to avoid overhead
    if len(completed_task_ids) < len(tasks_list):
        time.sleep(30)

pbar.close()
print("All export tasks have been processed.")

Exporting GLANCE Matrices:   0%|          | 0/19 [00:00<?, ?it/s]